In [1]:
import torch
import pandas as pd
import tarfile
import os
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [ ]:
# Get only patients that were on pressor medications
patient_meds = pd.read_csv('EPIC_EMR/EMR/patient_medications.csv', low_memory=False)
pressors = ['dopamine', 'vasopressin', 'dobutamine', 'epinephrine', 'norepinephrine']
pattern = '|'.join(pressors)
pressors_df = patient_meds[patient_meds['MEDICATION_NM'].str.contains(pattern, case=False, na=False)]
pressors_df = pressors_df.drop_duplicates()

In [ ]:
# Filter pressors data further
pattern = r'(?i)(lidocaine|nebu|.*-epinephrine|vasopressin infusion \(anesthesia\))'
pressors_df_filtered = pressors_df[~pressors_df['MEDICATION_NM'].str.contains(pattern)]

In [ ]:
# Get patients that were admitted to the ICU
patient_information = pd.read_csv('EPIC_EMR/EMR/patient_information.csv')
cols = ['LOG_ID', 'ICU_ADMIN_FLAG', 'SURGERY_DATE', 'DISCH_DISP_C', 'DISCH_DISP', 'BIRTH_DATE', 'SEX', 'PRIMARY_PROCEDURE_NM', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']
pressors_icu_df = pd.merge(pressors_df_filtered, patient_information[cols], on='LOG_ID', how='left')

# Get patients that only started on pressors after their surgery
pressors_icu_df['SURGERY_DATE'] = pd.to_datetime(pressors_icu_df['SURGERY_DATE'], format='%m/%d/%y %H:%M')
pressors_icu_df['START_DATE'] = pd.to_datetime(pressors_icu_df['START_DATE'])
pressors_icu_df_filtered = pressors_icu_df[pressors_icu_df['SURGERY_DATE'] < pressors_icu_df['START_DATE']]
pressors_icu_df_filtered = pressors_icu_df_filtered[pressors_icu_df_filtered['ICU_ADMIN_FLAG'] == 'Yes']

In [ ]:
# Correlate with primary diagnosis
patient_visit = pd.read_csv('EPIC_EMR/EMR/patient_visit.csv')
patient_visit = patient_visit.drop_duplicates()
pressors_diagnoses = pd.merge(pressors_icu_df_filtered, patient_visit[['LOG_ID', 'diagnosis_code', 'dx_name']], on='LOG_ID', how='left')

In [7]:
# Correlate with post-op complications
post_op_complications = pd.read_csv('EPIC_EMR/EMR/patient_post_op_complications.csv')
pressors_complications = pd.merge(pressors_diagnoses, post_op_complications[['LOG_ID', 'Element_abbr']], on='LOG_ID', how='left')
pressors_complications = pressors_complications.rename(columns={'Element_abbr': 'post_op_complication_type'})
pressors_complications

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,...,DISCH_DISP_C,DISCH_DISP,BIRTH_DATE,SEX,PRIMARY_PROCEDURE_NM,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,diagnosis_code,dx_name,post_op_complication_type
0,3,Hospital Encounter,6ec0099a158f7d6c,b4cecefb0cb0ec85,2021-02-08 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-02-08,...,20.0,Home Healthcare IP Admit Related,71.0,Female,"CATHETERIZATION, HEART, LEFT",2/4/21 23:54,2/16/21 21:45,NaN,NaN,AN Post-op Complications
1,3,Hospital Encounter,6ec0099a158f7d6c,b4cecefb0cb0ec85,2021-02-08 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-02-08,...,20.0,Home Healthcare IP Admit Related,71.0,Female,"CATHETERIZATION, HEART, LEFT",2/4/21 23:54,2/16/21 21:45,NaN,NaN,AN Post-op Complications
2,3,Hospital Encounter,6ec0099a158f7d6c,b4cecefb0cb0ec85,2021-02-08 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-02-08,...,20.0,Home Healthcare IP Admit Related,71.0,Female,"CATHETERIZATION, HEART, LEFT",2/4/21 23:54,2/16/21 21:45,NaN,NaN,AN Post-op Complications
3,3,Hospital Encounter,6ec0099a158f7d6c,b4cecefb0cb0ec85,2021-02-08 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-02-08,...,20.0,Home Healthcare IP Admit Related,71.0,Female,"CATHETERIZATION, HEART, LEFT",2/4/21 23:54,2/16/21 21:45,NaN,NaN,AN Post-op Complications
4,3,Hospital Encounter,6ec0099a158f7d6c,b4cecefb0cb0ec85,2021-02-08 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-02-08,...,20.0,Home Healthcare IP Admit Related,71.0,Female,"CATHETERIZATION, HEART, LEFT",2/4/21 23:54,2/16/21 21:45,NaN,NaN,AN Post-op Complications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7482485,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40,NaN,Coronary artery disease involving native coron...,AN Post-op Complications
7482486,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40,794.39,Abnormal stress test,AN Post-op Complications
7482487,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40,794.39,Abnormal stress test,AN Post-op Complications
7482488,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40,794.39,Abnormal stress test,AN Post-op Complications


In [38]:
pressors_complications.to_csv('pressors_final.csv')

In [39]:
pressors_complications['MEDICATION_NM'].unique()

array(['DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN',
       'NOREPINEPHRINE-SODIUM CHLORIDE 8-0.9 MG/250ML-% IV SOLN',
       'NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI)',
       'EPINEPHRINE 16 MG/250 ML INFUSION - CENTRAL LINE ONLY (UCI)',
       'VASOPRESSIN INFUSION (UCI)',
       'DOBUTAMINE 500 MG/250 ML D5W PREMIX INFUSION (UCI)',
       'EPINEPHRINE HCL (NASAL) 0.1 % NA SOLN',
       'DOPAMINE IN NS INFUSION (VARIABLE CONCENTRATION) (UCI)',
       'DOPAMINE 400 MG/250 ML D5W PREMIX INFUSION (UCI)',
       'NOREPINEPHRINE INFUSION (ANESTHESIA)',
       'VASOPRESSIN 20 UNIT/ML IJ SOLN',
       'EPINEPHRINE INFUSION (ANESTHESIA)',
       'DOBUTAMINE 500 MG/250 ML NS INFUSION (UCI)',
       'EPINEPHRINE PF 1 MG/10ML IJ SOSY',
       'EPINEPHRINE IRRIGATION FOR ARTHROSCOPY (OPTIME)',
       'EPINEPHRINE PF 1 MG/ML IJ SOLN',
       'EPINEPHRINE 1 MG IN 1000 ML NS TOPICAL SOLUTION (OPTIME)',
       'EPINEPHRINE HCL 1 MG/ML IJ SOLN',
       'norepinephrine (LEVOPHED) 4 mg in 250 mL NS 0.0

In [23]:
mortality = ['Expired', 'Coroner']
pattern = '|'.join(mortality)
mortality_patients = pressors_diagnoses[pressors_diagnoses['DISCH_DISP'].str.contains(pattern, case=False, na=False)]
mortality_patients['MRN'].nunique()

257

In [65]:
patient_history = pd.read_csv('EPIC_EMR/EMR/patient_history.csv')
patient_history[patient_history['mrn'] == 'b0064ab7d9de1fb0']

,mrn,diagnosis_code,dx_name
0,b0064ab7d9de1fb0,054.0,Eczema herpeticum
2017,b0064ab7d9de1fb0,365.11,Primary open angle glaucoma
4020,b0064ab7d9de1fb0,370.33,"Keratoconjunctivitis sicca, not specified as S..."
7870,b0064ab7d9de1fb0,V43.1,Pseudophakia
7910,b0064ab7d9de1fb0,362.60,Retinal degeneration
...,...,...,...
310983,b0064ab7d9de1fb0,379.21,Vitreous detachment of both eyes
316641,b0064ab7d9de1fb0,365.01,Open angle with borderline findings and low gl...
319565,b0064ab7d9de1fb0,NaN,"Chronic kidney disease, stage 5 (CMS-HCC)"
319585,b0064ab7d9de1fb0,NaN,"Chronic kidney disease, stage 4 (severe) (CMS-..."


In [5]:
pressors_diagnoses.to_csv('patient_medications_pressors.csv', index=False)

In [32]:
pressors_diagnoses['MRN'].nunique()

2334

In [49]:
patient_visit[patient_visit['mrn'] == '3dd05404ad8d1642']

,LOG_ID,mrn,diagnosis_code,dx_name
31524,469450c8ce421e1c,3dd05404ad8d1642,794.39,Abnormal stress test
32393,469450c8ce421e1c,3dd05404ad8d1642,NaN,Coronary artery disease involving native coron...
134432,c25832a2d87d7cdb,3dd05404ad8d1642,NaN,Coronary artery disease involving native coron...
148298,c25832a2d87d7cdb,3dd05404ad8d1642,794.39,Abnormal stress test


In [52]:
temp_patient = pressors_icu_df_filtered[pressors_icu_df_filtered['MRN'] == '3dd05404ad8d1642']
temp_patient[temp_patient['LOG_ID'] == 'c25832a2d87d7cdb']

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,...,MED_ROUTE_NM,ICU_ADMIN_FLAG,SURGERY_DATE,DISCH_DISP_C,DISCH_DISP,BIRTH_DATE,SEX,PRIMARY_PROCEDURE_NM,HOSP_ADMSN_TIME,HOSP_DISCH_TIME
597747,52,Anesthesia,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,Inpatient,452761,norepinephrine (LEVOPHED) 4 mg in sodium chlor...,NOREPINEPHRINE INFUSION (ANESTHESIA),2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
597748,52,Anesthesia,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,Inpatient,452761,norepinephrine (LEVOPHED) 4 mg in sodium chlor...,NOREPINEPHRINE INFUSION (ANESTHESIA),2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694656,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694657,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694658,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694659,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694660,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694661,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,ePrescribe,15982,DOBUTamine (DOBUTREX) premix infusion 1000 mg ...,DOBUTAMINE IN D5W 4-5 MG/ML-% IV SOLN,2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694662,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40
694663,3,Hospital Encounter,c25832a2d87d7cdb,3dd05404ad8d1642,2021-05-26 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-05-26,...,IntraVENOUS,Yes,2021-05-21,20.0,Home Healthcare IP Admit Related,60.0,Male,"CATHETERIZATION, HEART, LEFT, WITH INTERVENTIO...",5/21/21 6:25,5/31/21 10:40


In [26]:
post_op_complications = pd.read_csv('EPIC_EMR/EMR/patient_post_op_complications.csv')
post_op_complications

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE
0,b00c588406aae5cc,1a42f0bed60c5598,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
1,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
2,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,AN AQI POST-OP COMPLICATIONS,ORDER,AN Post-op Complications,NaN
4,2ae43a4a39735956,59c5483e0654e105,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,Other
...,...,...,...,...,...,...
203940,41e9d0fc85705586,a7b31506e6207021,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203941,c998b4d14118fdf8,3f1772becaaebb81,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203942,689a22b76d123ff6,65c32dd7fd5e35e7,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203943,58a3296cc7491dd2,f7fc111bcf19de88,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN


In [31]:
len(set(pressors_diagnoses['MRN']).intersection(post_op_complications['MRN']))

2309

In [28]:
post_op_complications['SMRTDTA_ELEM_VALUE'].unique()

array([nan, 'Other', 'Airway', 'Oral mucosal injury', 'Metabolic',
       'Laryngospasm (SaO2<90%)', 'Pulmonary Embolism',
       'Patient aspirated upon emergence requiring reintubation and transfer to ICU.',
       'Administrative', 'Hypotension (SBP<80 for 10 min)',
       'Cardiovascular',
       'Patient had hypotensive reaction to one unit of PRBCs. Documented in chart',
       'Neurological', 'Instability (hemodynamic)',
       'Delerium (postoperative)', 'Arrest (cardiac)',
       'Ischemia (myocardial)',
       'machine check and after patient arrived in the room.',
       'Hypoxemia (SaO2<90% or PaO2<60mmHg)', 'Respiratory',
       'Failed Intubation', 'sed by anesthesiology team and cardiology',
       'Hypertension', 'Hypoxemia (SaO2<90%)',
       'ted overnight post procedure. Care at this time includes drops and Eryth. Oint for possible corneal abrasion.      Ophth stopped by in PACU to see patient. No evidence of abrasions and eye is dilated for examination later today. 

## Waveform Data

In [13]:
waveform_to_mrn = pd.read_csv('mover_data/mover-download.ics.uci.edu/mover-download.ics.uci.edu/EPIC_MRN_PAT_ID.csv')

# get patient IDs for pressor patients
set_pressors = set(pressors_icu_df_filtered['MRN'])
filtered_waveforms = waveform_to_mrn[waveform_to_mrn['MRN'].isin(set_pressors)]
patient_ids = set(filtered_waveforms['PAT_ID'])

# use patient IDs to get all waveform filepaths for those patients
directories = ['ART_EPIC/EPIC_07_22', 'ART_EPIC/EPIC_08_10', 'ART_EPIC/EPIC_11_28']
filepaths = []
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith('.csv') and any(sub in os.path.splitext(filename)[0] for sub in patient_ids):
            file_path = os.path.join(directory, filename)
            filepaths.append(file_path)
        
print(len(filepaths))

260


In [15]:
directories = [
    'mover_data/mover-download.ics.uci.edu/wave_1',
    'mover_data/mover-download.ics.uci.edu/wave_2',
    'mover_data/mover-download.ics.uci.edu/wave_3'
]
filepaths = []
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith('.csv') and any(sub in os.path.splitext(filename)[0] for sub in patient_ids):
            file_path = os.path.join(directory, filename)
            filepaths.append(file_path)
            
print(len(filepaths))

2993


In [23]:
valid_paths = []
usecols = ['datapoints']

for filepath in filepaths:
    df = pd.read_csv(filepath, usecols=usecols, nrows=1)
    data = map(int, df.loc[0, 'datapoints'].split('|'))
    if all(num >= 0 for num in data):
        valid_paths.append(filepath)
        print(f'adding filepath {filepath}')

print(len(valid_paths))

adding filepath mover_data/mover-download.ics.uci.edu/wave_3/58bab13924c88374IP.csv
1
